# Analisis de las variables numericas de cada cluster

Gracias al analisis en el notebook anterior sabemos que las diferencias entre los 12 grupos (clusters) no radican en las variables categoricas sino mas bien todo parece indicar que estas diferencias radican en las variables numericas que son las que analizaremos en este notebook 

In [3]:
# Importamos las librerías necesarias
import pandas as pd
from scipy.stats import f_oneway, levene, kruskal, shapiro
import os

df_customer = pd.read_csv("dataset/segmented_customers.csv")
# df_customer['Cluster'] = df_customer['Cluster'].astype('category')

# Obtenemos la lista de variables numéricas relevantes para nuestro análisis
numerical_cols = ['Age', 'Purchase Amount (USD)', 'Review Rating', 'Previous Purchases']
num_clusters = sorted(df_customer['Cluster'].unique())

df_customer.head()

,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Payment Method,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Preferred Payment Method,Frequency of Purchases,Cluster
0,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Credit Card,Express,Yes,Yes,14,Venmo,Fortnightly,2
1,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Bank Transfer,Express,Yes,Yes,2,Cash,Fortnightly,1
2,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Cash,Free Shipping,Yes,Yes,23,Credit Card,Weekly,2
3,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,PayPal,Next Day Air,Yes,Yes,49,PayPal,Weekly,2
4,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Cash,Free Shipping,Yes,Yes,31,PayPal,Annually,2


Para poder afirmar que, por ejemplo, el promedio de la variable Age en el Cluster 1 es mayor que el promedio de Age en el Cluster 8, no podemos basarnos simplemente en la comparacion de las medias. Debemos ir mas alla y asegurarnos de que esta diferencia no es producto del azar. Es por ello que vamos a realizar pruebas estadisticas para verificar si esta diferencia realmente existe.

Una de las pruebas más comunes es el Analisis de Varianza (ANOVA). Sin embargo, para que los resultados de ANOVA sean confiables, se deben cumplir una serie de requisitos en los datos:

* Normalidad: Cada variable debe seguir una distribución normal dentro de cada clúster.

* Homogeneidad de Varianzas: Debe existir homogeneidad entre las varianzas de los clústeres para cada variable.

A continuacion, vamos a verificar si se cumplen estos supuestos. Los resultados se presentarán en una tabla clara para que podamos decidir si ANOVA es la prueba adecuada o si debemos recurrir a una alternativa.

In [4]:
print("\n" + "="*80)
print("TABLA DE VERIFICACIÓN DE SUPUESTOS DE NORMALIDAD (Shapiro-Wilk)")
print("   - Filas: Clústeres. Columnas: Variables.")
print("   - 'Sí pasa' significa que el p-valor >= 0.05 y el clúster sigue una distribución normal.")
print("="*80)

# Creamos una lista de diccionarios para almacenar los resultados por clúster
shapiro_results_list = []

# Iteramos sobre cada clúster para realizar las pruebas de Shapiro-Wilk
for cluster_id in num_clusters:
    cluster_row = {'Cluster ID': f'Clúster {cluster_id}'}
    for col in numerical_cols:
        cluster_data = df_customer[df_customer['Cluster'] == cluster_id][col].dropna()
        if len(cluster_data) >= 3: # Shapiro-Wilk requiere al menos 3 observaciones
            shapiro_stat, shapiro_p = shapiro(cluster_data)
            shapiro_result = "Sí pasa" if shapiro_p >= 0.05 else f"No pasa (p={shapiro_p:.4f})"
        else:
            shapiro_result = "N/A (pocas obs.)"
        cluster_row[col] = shapiro_result
    shapiro_results_list.append(cluster_row)

# Convertimos la lista de diccionarios a un DataFrame para mostrar como tabla
shapiro_df = pd.DataFrame(shapiro_results_list)
print(shapiro_df.to_markdown(index=False))

print("\n" + "="*80)
print("TABLA DE VERIFICACIÓN DE SUPUESTOS DE HOMOGENEIDAD DE VARIANZAS (Levene)")
print("   - Se compara la varianza de la variable entre TODOS los clústeres.")
print("   - 'Sí pasa' significa que el p-valor >= 0.05 y las varianzas son homogéneas.")
print("="*80)

levene_results_list = []
for col in numerical_cols:
    cluster_groups = [df_customer[df_customer['Cluster'] == i][col].dropna() for i in num_clusters]
    try:
        levene_stat, levene_p = levene(*cluster_groups)
        levene_result = "Sí pasa" if levene_p >= 0.05 else f"No pasa (p={levene_p:.4f})"
    except ValueError:
        levene_result = "N/A (error en grupos)"
    levene_results_list.append({'Variable': col, 'Prueba de Levene': levene_result})
    
levene_df = pd.DataFrame(levene_results_list)
print(levene_df.to_markdown(index=False))
print("\n" + "="*80)


TABLA DE VERIFICACIÓN DE SUPUESTOS DE NORMALIDAD (Shapiro-Wilk)
   - Filas: Clústeres. Columnas: Variables.
   - 'Sí pasa' significa que el p-valor >= 0.05 y el clúster sigue una distribución normal.
| Cluster ID   | Age                | Purchase Amount (USD)   | Review Rating      | Previous Purchases   |
|:-------------|:-------------------|:------------------------|:-------------------|:---------------------|
| Clúster 0    | No pasa (p=0.0009) | No pasa (p=0.0009)      | No pasa (p=0.0000) | No pasa (p=0.0007)   |
| Clúster 1    | No pasa (p=0.0000) | No pasa (p=0.0000)      | No pasa (p=0.0000) | No pasa (p=0.0000)   |
| Clúster 2    | No pasa (p=0.0000) | No pasa (p=0.0000)      | No pasa (p=0.0000) | No pasa (p=0.0000)   |
| Clúster 3    | No pasa (p=0.0000) | No pasa (p=0.0000)      | No pasa (p=0.0000) | No pasa (p=0.0000)   |
| Clúster 4    | No pasa (p=0.0000) | No pasa (p=0.0000)      | No pasa (p=0.0000) | No pasa (p=0.0000)   |
| Clúster 5    | No pasa (p=0.0000) | No pa

Para mi sopresa solo uno de los requisitos se cumplio el de HOMOGENEIDAD DE VARIANZAS (Levene) y el de SUPUESTOS DE NORMALIDAD (Shapiro-Wilk) no lo hizo indicado que las variables numericas de todos los clusters no siguen una distribucion normal

Afortunadamente la estadistica inferencial nos dota de muchas herramientas para el proposito que estamos buscando y una de ellas es la prueba de Kruskall Wallis la cual aplicaremos ahora para saber si existen diferencias entre los distintos grupos con las variables numericas que estamos tratando

In [5]:
print("\n" + "="*80)
print("TABLA DE RESULTADOS DE LA PRUEBA DE KRUSKAL-WALLIS")
print("   - Se determina qué variables tienen distribuciones significativamente diferentes.")
print("   - Un P-valor < 0.05 indica una diferencia significativa.")
print("="*80)

# Lista para almacenar los resultados de la prueba de Kruskal-Wallis
kruskal_results_list = []

for col in numerical_cols:
    # Creamos una lista de arrays con los datos de cada clúster
    cluster_groups = [df_customer[df_customer['Cluster'] == i][col].dropna() for i in num_clusters]
    
    # Verificamos si los grupos tienen suficientes datos
    if any(len(group) < 5 for group in cluster_groups):
        kruskal_result = "N/A (grupos pequeños)"
        p_value = "N/A"
    else:
        try:
            kruskal_stat, kruskal_p_value = kruskal(*cluster_groups)
            p_value = f"{kruskal_p_value:.4f}"
            kruskal_result = "Sí pasa" if kruskal_p_value < 0.05 else "No pasa"
        except ValueError:
            kruskal_result = "N/A (error)"
            p_value = "N/A"
            
    kruskal_results_list.append({
        'Variable': col,
        'P-valor': p_value,
        'Diferencia Signif. (p<0.05)': kruskal_result
    })
    
# Convertimos la lista de diccionarios a un DataFrame para mostrar como tabla
kruskal_df = pd.DataFrame(kruskal_results_list)
print(kruskal_df.to_markdown(index=False, numalign="left", stralign="left"))
print("\n" + "="*80)


TABLA DE RESULTADOS DE LA PRUEBA DE KRUSKAL-WALLIS
   - Se determina qué variables tienen distribuciones significativamente diferentes.
   - Un P-valor < 0.05 indica una diferencia significativa.
| Variable              | P-valor   | Diferencia Signif. (p<0.05)   |
|:----------------------|:----------|:------------------------------|
| Age                   | 0.3827    | No pasa                       |
| Purchase Amount (USD) | 0.2342    | No pasa                       |
| Review Rating         | 0.5902    | No pasa                       |
| Previous Purchases    | 0.2247    | No pasa                       |



Gracias a la prueba Kruskall Wallis ahora sabemos que no existen diferencias significativas entre las variables numericas de nuestros clusters y como se mostro en el jupiter anterior ya que ninguna de las variables categoricas es una caracteristica distintiva de estos, solo hay una forma de caracterizar estos cluster (grupos de clientes) la cual es realizar un analisis bivariado para asi poder obtener esas caracteristicas distintivas de cada cluster que estamos buscando

Y ese sera nuetro siguiente paso a realizar el cual ejecutaremos en los siguientes notebooks